In [35]:
import pysam
import pandas as pd
import os

data_path = 'data/HW4/'

reads1 = f'{data_path}ecoli_400K_err_1.fastq'
reads2 = f'{data_path}ecoli_400K_err_2.fastq'
ref_genome = f'{data_path}MG1655-K12.first400K.fasta'

spades_res = f'{data_path}spades_test/'
spades_cor_path = f'{spades_res}/corrected/'
spades_r_1 = f'{spades_cor_path}ecoli_400K_err_1.00.0_0.cor.fastq.gz'
spades_r_2 = f'{spades_cor_path}ecoli_400K_err_2.00.0_0.cor.fastq.gz'

bbduk_r_1 = f'{data_path}bbduk_tes_r_1p.fq'
bbduk_r_2 = f'{data_path}bbduk_tes_r_2p.fq'
bbduk_r_1u = f'{data_path}bbduk_tes_r_1u.fq'
bbduk_r_2u = f'{data_path}bbduk_tes_r_2u.fq'

aln = f'{data_path}test_aln.sam'
aln_spades = f'{data_path}test_aln_spades.sam'
aln_bbduk = f'{data_path}test_aln_bbduk.sam'

aln_bam = f'{data_path}test_aln.bam'
aln_spades_bam = f'{data_path}test_aln_spades.bam'
aln_bbduk_bam = f'{data_path}test_aln_bbduk.bam'

aln_bam_sorted = f'{data_path}test_aln.sorted.bam'
aln_spades_bam_sorted = f'{data_path}test_aln_spades.sorted.bam'
aln_bbduk_bam_sorted = f'{data_path}test_aln_bbduk.sorted.bam'

# 1. Исправление ошибок

Для исправления ошибок будем использовать BayesHammer and Trimmomatic

In [22]:
!conda run spades.py -1 {reads1} -2 {reads2} --only-error-correction -o {spades_res}



== Warning ==  No assembly mode was specified! If you intend to assemble high-coverage multi-cell/isolate data, use '--isolate' option.


Command line: /Users/nikitakonstantinovskiy/.conda/envs/HW_NGS/bin/spades.py	-1	/Users/nikitakonstantinovskiy/Documents/Bioinformatics/IB/NGS/HW_NGS/data/HW4/ecoli_400K_err_1.fastq	-2	/Users/nikitakonstantinovskiy/Documents/Bioinformatics/IB/NGS/HW_NGS/data/HW4/ecoli_400K_err_2.fastq	--only-error-correction	-o	/Users/nikitakonstantinovskiy/Documents/Bioinformatics/IB/NGS/HW_NGS/data/HW4/spades_test	

System information:
  SPAdes version: 3.15.2
  Python version: 3.8.8
  OS: macOS-10.15.4-x86_64-i386-64bit

Output dir: /Users/nikitakonstantinovskiy/Documents/Bioinformatics/IB/NGS/HW_NGS/data/HW4/spades_test
Mode: ONLY read error correction (without assembling)
Debug mode is turned OFF

Dataset parameters:
  Standard mode
  For multi-cell/isolate data we recommend to use '--isolate' option; for single-cell MDA data use '--sc'; for metagenomic data us

In [23]:
!java -jar utils/Trimmomatic-0.39/trimmomatic-0.39.jar PE -phred33 {reads1} {reads2} {bbduk_r_1} {bbduk_r_1u} \
{bbduk_r_2} {bbduk_r_2u} LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20

TrimmomaticPE: Started with arguments:
 -phred33 data/HW4/ecoli_400K_err_1.fastq data/HW4/ecoli_400K_err_2.fastq data/HW4/bbduk_tes_r_1p.fq data/HW4/bbduk_tes_r_1u.fq data/HW4/bbduk_tes_r_2p.fq data/HW4/bbduk_tes_r_2u.fq LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20
Input Read Pairs: 1381602 Both Surviving: 1344536 (97,32%) Forward Only Surviving: 23598 (1,71%) Reverse Only Surviving: 10815 (0,78%) Dropped: 2653 (0,19%)
TrimmomaticPE: Completed successfully


# 2. Выравнивание

In [24]:
!conda run bwa index {ref_genome}

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.06 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.02 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index data/HW4/MG1655-K12.first400K.fasta
[main] Real time: 0.091 sec; CPU: 0.089 sec



In [36]:
!bwa mem {ref_genome} {reads1} {reads2} > {aln}
!bwa mem {ref_genome} {spades_r_1} {spades_r_2} > {aln_spades}
!bwa mem {ref_genome} {bbduk_r_1} {bbduk_r_2} > {aln_bbduk}

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 100000 sequences (10000000 bp)...
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48301, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.27, 10.16)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 2.315 CPU sec, 2.271 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48357, 0, 0)
[M::mem_pestat] skip orientat

[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48231, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.27, 10.13)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 2.533 CPU sec, 2.384 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48187, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orienta

[M::mem_process_seqs] Processed 100000 reads in 2.270 CPU sec, 2.127 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48350, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.35, 10.15)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 2.308 CPU sec, 2.157 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48319, 0, 0)
[M::mem_pestat] skip orientation FF as there are n

[M::mem_process_seqs] Processed 100000 reads in 2.029 CPU sec, 1.749 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48317, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 221)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (179, 249)
[M::mem_pestat] mean and std.dev: (214.24, 10.15)
[M::mem_pestat] low and high boundaries for proper pairs: (165, 263)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 2.062 CPU sec, 1.786 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48419, 0, 0)
[M::mem_pestat] skip orientation FF as there are n

[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48421, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.38, 10.13)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 2.028 CPU sec, 1.746 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48381, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orienta

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 110200 sequences (10000110 bp)...
[M::process] read 110814 sequences (10000040 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 53018, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.25, 10.17)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 110200 reads in 2.008 CPU sec, 1.959 real sec
[M::process] read 111006 sequences (10000050 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 53373, 0, 0)
[M::mem_pestat] skip orientat

[M::process] read 110288 sequences (10000060 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 54846, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.37, 10.15)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 114452 reads in 2.203 CPU sec, 2.026 real sec
[M::process] read 109970 sequences (10000159 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 53098, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orienta

[M::mem_process_seqs] Processed 106892 reads in 2.065 CPU sec, 1.903 real sec
[M::process] read 44036 sequences (4155865 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 51260, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.31, 10.11)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 106614 reads in 2.040 CPU sec, 1.909 real sec
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 21248, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size 

In [37]:
!samtools flagstat {aln}

2763204 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2762617 + 0 mapped (99.98% : N/A)
2763204 + 0 paired in sequencing
1381602 + 0 read1
1381602 + 0 read2
2760820 + 0 properly paired (99.91% : N/A)
2762132 + 0 with itself and mate mapped
485 + 0 singletons (0.02% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [38]:
!samtools flagstat {aln_spades}

2716052 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2715942 + 0 mapped (100.00% : N/A)
2716052 + 0 paired in sequencing
1358026 + 0 read1
1358026 + 0 read2
2714584 + 0 properly paired (99.95% : N/A)
2715874 + 0 with itself and mate mapped
68 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [39]:
!samtools flagstat {aln_bbduk}

2689072 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2687824 + 0 mapped (99.95% : N/A)
2689072 + 0 paired in sequencing
1344536 + 0 read1
1344536 + 0 read2
2685506 + 0 properly paired (99.87% : N/A)
2686782 + 0 with itself and mate mapped
1042 + 0 singletons (0.04% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [40]:
!samtools view -b {aln} > {aln_bam}
!samtools sort {aln_bam} > {aln_bam_sorted}
!samtools index {aln_bam_sorted}

!samtools view -b {aln_spades} > {aln_spades_bam}
!samtools sort {aln_spades_bam} > {aln_spades_bam_sorted}
!samtools index {aln_spades_bam_sorted}

!samtools view -b {aln_bbduk} > {aln_bbduk_bam}
!samtools sort {aln_bbduk_bam} > {aln_bbduk_bam_sorted}
!samtools index {aln_bbduk_bam_sorted}

[bam_sort_core] merging from 1 files and 1 in-memory blocks...


# 3. Оценка качества исправления ошибок

In [41]:
def aln_err(line):
    ref_seq = line.get_reference_sequence()
    aln_seq = line.query_sequence
    aln_seq = aln_seq[line.query_alignment_start: line.query_alignment_end]
    r_ind = 0
    a_ind = 0

    err = []
    
    for ct in line.cigartuples:
        if ct[0] == 0:
            for i in range(ct[1]):
                if ref_seq[r_ind].upper() != aln_seq[a_ind].upper():
                    err.append(0)
                else:
                    err.append(1)
                r_ind += 1
                a_ind += 1
        elif ct[0] == 1:
            a_ind += ct[1]
        elif ct[0] == 2:
            r_ind += ct[1]
    
    return err

def errors(orig_aln, err_cor_aln):
    bamfile_orig = pysam.AlignmentFile(orig_aln, 'rb')
    bamfile_err_cor = pysam.AlignmentFile(err_cor_aln, 'rb')
    
    ref_name_orig = bamfile_orig.get_reference_name(0)

    bamfile_err_cor_ind = pysam.IndexedReads(bamfile_err_cor)
    bamfile_err_cor_ind.build()
    
    errors = {
        'Error in raw data': {'Error in corrected reads': 0, 'Correct base in corrected reads': 0, 'Base is absent in corrected reads': 0},
        'Correct base in raw data': {'Error in corrected reads': 0, 'Correct base in corrected reads': 0, 'Base is absent in corrected reads': 0},
    }
        
    for line in bamfile_orig.fetch(ref_name_orig):
        read_type = line.is_read1
        line_cor = None
        try:
            for lcor in bamfile_err_cor_ind.find(line.query_name):
                if read_type == lcor.is_read1:
                    line_cor = lcor
                    break
        except:
            continue
        
        if line.cigartuples is not None:
            err = aln_err(line)
        else:
            err = []
        
        if line_cor is not None and line_cor.cigartuples is not None:
            err_cor = aln_err(line_cor)
        else:
            err_cor = []        
        
        i_err_cor = line.reference_start - line_cor.reference_start
        
        for i_err in range(len(err)):
            raw_data = 'Error in raw data' if err[i_err] == 0 else 'Correct base in raw data'
            cor_data = ''
            
            if i_err_cor < 0 or i_err_cor >= len(err_cor):
                cor_data = 'Base is absent in corrected reads'
            else:
                cor_data = 'Error in corrected reads' if err_cor[i_err_cor] == 0 else 'Correct base in corrected reads'
            
            errors[raw_data][cor_data] += 1
            
            i_err_cor += 1
        
    return pd.DataFrame(errors).T

# BayesHammer Results

In [42]:
errors(aln_bam_sorted, aln_spades_bam_sorted)

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,247788,832542,1200515
Correct base in raw data,381100,246955705,16187708


# Trimmomatic Results

In [43]:
errors(aln_bam_sorted, aln_bbduk_bam_sorted)

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,886504,1009,1344256
Correct base in raw data,614,241129878,19807902
